## Import All Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

## Read Wikipedia Page to get the table

In [2]:
import requests
#Read the page
url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url_wiki, header=0)

neighborhoods = dfs[0].copy()
neighborhoods.head()
#neighborhoods.count()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Remove 'Not assigned' cases

In [3]:
#Remove 'Not assigned' cases

print('No. of Neighborhoodes before processing: ', len(neighborhoods))

dropIndex = neighborhoods[neighborhoods['Borough'] == 'Not assigned'].index
neighborhoods.drop(index = dropIndex, inplace = True)

neighborhoods.Neighbourhood[neighborhoods['Neighbourhood'] == 'Not assigned'] = neighborhoods.Borough

print('No. of Neighborhoodes after processing: ', len(neighborhoods))
neighborhoods.reset_index(inplace=True)
neighborhoods.drop('index', axis = 1, inplace=True)
neighborhoods.head(5)

No. of Neighborhoodes before processing:  288
No. of Neighborhoodes after processing:  211


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Group Neighborhoods

In [4]:
#Grouped Neighborhoods
g_neighborhoods = neighborhoods.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ', '.join(x)).reset_index()
g_neighborhoods.rename(index=str, columns={"Postcode": "Postal Code", "Neighbourhood": "Neighborhood"}, inplace=True)
g_neighborhoods.head(5)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Size of the grouped Neighborhoods

In [5]:
g_neighborhoods.shape

(103, 3)

In [6]:
g_neighborhoods

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Merge and display Neighborhood data with latitude and longitude

## Download & Read Geospatial Data

In [7]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data

print('Data downloaded!')

geo_df = pd.read_csv('Geospatial_data.csv')
geo_df.head()

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge and display Neighborhood data with latitude and longitude

In [8]:
df_neighborhoods = pd.merge(g_neighborhoods, geo_df, how='outer', on='Postal Code')
df_neighborhoods.head(5)
#df_neighborhoods[df_neighborhoods['Borough'].str.contains('Toronto') == True]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Plot boroughs that contain the word "Toronto"

## Use geopy library to get the latitude and longitude values of "Toronto"

In [9]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with all neighborhoods superimposed on top

In [11]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Identify Boroughs that contain the word Toronto

In [12]:
df_neighborhoods['Borough'].value_counts() #All Neighborhoods
Toronto_data = df_neighborhoods[df_neighborhoods['Borough'].str.contains('Toronto') == True] #To be used further
print("Boroughs containing word Toronto with neighborhood  count :\n", Toronto_data['Borough'].value_counts()) #Filtered ones with Toronto

#List of unique Boroughs
Borough_list = Toronto_data['Borough'].unique()
print("Unique Borough List : ", Borough_list)

Boroughs containing word Toronto with neighborhood  count :
 Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64
Unique Borough List :  ['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


## Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'YXO1CNCNUXNDNQXBKVPFQSKYLUN21BXFCIWFNHHQ2AF40QDT' # your Foursquare ID
CLIENT_SECRET = 'MFVPCBLEV0XDB3XIWJESJPLNLIIONZFOR541CZ4IVAQWD51S' # your Foursquare Secret
VERSION = '20180605'
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YXO1CNCNUXNDNQXBKVPFQSKYLUN21BXFCIWFNHHQ2AF40QDT
CLIENT_SECRET:MFVPCBLEV0XDB3XIWJESJPLNLIIONZFOR541CZ4IVAQWD51S


#### Create function that extracts the category of the venue

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Explore Neighborhoods in Toronto

#### Create a function to repeat the same process to all the neighborhoods in Toronto

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called Toronto_venues

In [18]:
#Get Nearby venues
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

print("All Venues read....")

print(Toronto_venues.shape)
Toronto_venues.head()

All Venues read....
(1702, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [19]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))
TCS = Toronto_venues[Toronto_venues['Venue Category'].str.contains('Coffee') == True]
TCS.reset_index(inplace = True, drop=True)
Toronto_venues = TCS

There are 239 uniques categories.


#### Check how many venues and unique categories were returned for each neighborhood

In [20]:
print("Venues : \n") 
Toronto_venues.groupby('Neighborhood').count()

Venues : 



,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",8,8,8,8,8,8
Berczy Park,4,4,4,4,4,4
"Brockton, Exhibition Place, Parkdale Village",2,2,2,2,2,2
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",1,1,1,1,1,1
"Cabbagetown, St. James Town",4,4,4,4,4,4
Central Bay Street,12,12,12,12,12,12
"Chinatown, Grange Park, Kensington Market",3,3,3,3,3,3
Christie,1,1,1,1,1,1
Church and Wellesley,6,6,6,6,6,6


In [21]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 1 uniques categories.


In [22]:
Toronto_venues['Venue Category'].unique()

array(['Coffee Shop'], dtype=object)

### Analyze Each Neighborhood

In [23]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Coffee Shop
0,"The Danforth West, Riverdale",1
1,"The Danforth West, Riverdale",1
2,"The Danforth West, Riverdale",1
3,"The Danforth West, Riverdale",1
4,Studio District,1


In [24]:
Toronto_onehot.shape

(144, 2)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Coffee Shop
0,"Adelaide, King, Richmond",1
1,Berczy Park,1
2,"Brockton, Exhibition Place, Parkdale Village",1
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",1
4,"Cabbagetown, St. James Town",1
5,Central Bay Street,1
6,"Chinatown, Grange Park, Kensington Market",1
7,Christie,1
8,Church and Wellesley,1
9,"Commerce Court, Victoria Hotel",1


In [26]:
Toronto_grouped.shape

(27, 2)

#### Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print('\033[1m' + "----"+hood+"----" + '\033[0m')
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
   # print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop   1.0
----Berczy Park----
         venue  freq
0  Coffee Shop   1.0
----Brockton, Exhibition Place, Parkdale Village----
         venue  freq
0  Coffee Shop   1.0
----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
         venue  freq
0  Coffee Shop   1.0
----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop   1.0
----Central Bay Street----
         venue  freq
0  Coffee Shop   1.0
----Chinatown, Grange Park, Kensington Market----
         venue  freq
0  Coffee Shop   1.0
----Christie----
         venue  freq
0  Coffee Shop   1.0
----Church and Wellesley----
         venue  freq
0  Coffee Shop   1.0
----Commerce Court, Victoria Hotel----
         venue  freq
0  Coffee Shop   1.0
----Davisville----
         venue  freq
0  Coffee Shop   1.0
----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
         venue  freq
0  Co

#### Neighborhoods with Coffee Shops

In [28]:
#Neighborhoods with only Coffee Shops

num_top_venues = 5
Toronto_grouped.head()
Toronto_CoffeeShop_grouped = Toronto_grouped[['Neighborhood', 'Coffee Shop']].copy()
Toronto_CoffeeShop_grouped = Toronto_CoffeeShop_grouped[Toronto_CoffeeShop_grouped['Coffee Shop']>0].reset_index(drop = True)
Toronto_CoffeeShop_grouped
Toronto_CoffeeShop_grouped.sort_values(by = 'Coffee Shop', ascending = False, axis = 0, inplace = True)
Toronto_CoffeeShop_grouped.reset_index(drop = True, inplace=True)
Toronto_CoffeeShop_grouped['Neighborhood']


0                              Adelaide, King, Richmond
1                First Canadian Place, Underground city
2                   The Annex, North Midtown, Yorkville
3                                       Studio District
4                       Stn A PO Boxes 25 The Esplanade
5                                        St. James Town
6                              Ryerson, Garden District
7                                    Runnymede, Swansea
8                                Parkdale, Roncesvalles
9                                    North Toronto West
10                             Little Portugal, Trinity
11                            Harbourfront, Regent Park
12    Harbourfront East, Toronto Islands, Union Station
13                         Dovercourt Village, Dufferin
14                                          Berczy Park
15             Design Exchange, Toronto Dominion Centre
16    Deer Park, Forest Hill SE, Rathnelly, South Hi...
17                                           Dav

In [30]:
# Sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
1,Berczy Park,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
4,"Cabbagetown, St. James Town",Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop


# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Remove all NA cases
Toronto_merged.dropna(inplace=True)

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop


Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [35]:
## Define a method to show Specific Characteristics of the cluster
def Cluster_Details(Cluster, Filter, Criteria) :
    l1 = len(Cluster)
    l2 = 0
    Bors = Cluster['Borough']
    Cluster.drop(['Cluster Labels'], axis=1, inplace=True)
    if Cluster.empty == False :
        Cluster = Cluster[Cluster.apply(lambda row: row.astype(str).str.contains(Filter).any(), axis=1)]
        Cluster = Cluster.transpose()
        if Cluster.empty == False :
            #print(Cluster)
            Cluster = Cluster[Cluster.apply(lambda row: row.astype(str).str.contains(Filter).any(), axis=1)]
            Cluster = Cluster.transpose()
            l2 = len(Cluster)              
            Cluster = pd.merge(Cluster, Bors, left_index=True, right_index=True)
            
    
    print("Out of", l1, "Neighborhoods in the Cluster", l2, "Neighborhoods have", Filter, Criteria)  
    if l2 >0 :
        #print("Selected Neighborhoods are:", Cluster['Borough'].to_string(index=False))
        print("Selected Boroughs with No. of Neighborhoods are:\n", Cluster['Borough'].value_counts().to_string())
    
    return Cluster

#### Cluster 1

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
43,East Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
46,Central Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
47,Central Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
49,Central Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
51,Downtown Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
52,Downtown Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
53,Downtown Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
54,Downtown Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop
55,Downtown Toronto,0.0,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop,Coffee Shop


#### Cluster 2

In [37]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 3

In [38]:


Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [39]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 5

In [40]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [41]:
for n in range(5):
    #####print("\nCluster", n+1, ":")
    print('\033[1m' + "---- Cluster "+ str(n+1) + "----" + '\033[0m')
    Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == n, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()
    Cluster = Cluster_Details(Cluster, 'Coffee', 'Shop')
    print('\n')


---- Cluster 1----
Out of 27 Neighborhoods in the Cluster 27 Neighborhoods have Coffee Shop
Selected Boroughs with No. of Neighborhoods are:
 Downtown Toronto    16
West Toronto         5
Central Toronto      4
East Toronto         2


---- Cluster 2----
Out of 0 Neighborhoods in the Cluster 0 Neighborhoods have Coffee Shop


---- Cluster 3----
Out of 0 Neighborhoods in the Cluster 0 Neighborhoods have Coffee Shop


---- Cluster 4----
Out of 0 Neighborhoods in the Cluster 0 Neighborhoods have Coffee Shop


---- Cluster 5----
Out of 0 Neighborhoods in the Cluster 0 Neighborhoods have Coffee Shop


